In [8]:
import numpy as np
from pathlib import Path
import polars as pl
from pprint import pprint

from crosssections import create_crosssection_catalog, load_crosssections_into_array, load_crosssections_into_Dataset

In [2]:
catalog = create_crosssection_catalog("/Volumes/ResearchStorage/Opacities_0v10/gases")

/Users/arthur/Documents/Astronomy/2019/Retrieval/Code/newAPOLLO/crosssections.py:91: UserWarning: There is apparently more than one set of header info. Check to see all have the same pressure, temperature, and wavelength ranges, and/or resolutions.
  warn("There is apparently more than one set of header info. " +


In [9]:
pprint(catalog)

{'50k': CrossSectionDirectory(number_of_pressure_layers=18,
                              minimum_log_pressure=-6.0,
                              maximum_log_pressure=2.5,
                              number_of_temperatures=36,
                              minimum_log_temperature=1.875061,
                              maximum_log_temperature=3.599199,
                              number_of_spectral_elements=73318,
                              minimum_wavelength=0.6,
                              maximum_wavelength=2.6,
                              effective_resolution=50000.0,
                              name='50k',
                              species=('ch4',
                                       'co',
                                       'co2',
                                       'crh',
                                       'feh',
                                       'h2he',
                                       'h2o',
                                       'h2onl

In [3]:
test_directory = catalog["nir"]

In [10]:
opacity_Dataset = load_crosssections_into_Dataset(test_directory)

In [11]:
opacity_Dataset.wavelengths.attrs.update(dict(units="micron"))

In [15]:
opacity_Dataset.pressures.attrs.update(dict(units="dex(mbar)"))

In [16]:
opacity_Dataset.temperatures.attrs.update(dict(units="K"))

In [ ]:
#nc_save_file = "nir_opacities.nc"
#opacity_Dataset.to_netcdf(nc_save_file)

In [17]:
opacity_Dataset

<xarray.Dataset>
Dimensions:       (pressures: 18, temperatures: 36, wavelengths: 21204)
Coordinates:
  * pressures     (pressures) float64 -3.0 -2.5 -2.0 -1.5 ... 4.0 4.5 5.0 5.5
  * temperatures  (temperatures) float64 75.0 84.01 94.11 ... 3.553e+03 3.98e+03
  * wavelengths   (wavelengths) float64 0.6 0.6001 0.6001 ... 4.999 5.0 5.0
Data variables: (12/19)
    Burrows_alk   (pressures, temperatures, wavelengths) float64 3.01e-20 ......
    ch4           (pressures, temperatures, wavelengths) float64 3.131e-24 .....
    co            (pressures, temperatures, wavelengths) float64 1.795e-20 .....
    co2           (pressures, temperatures, wavelengths) float64 3.346e-20 .....
    crh           (pressures, temperatures, wavelengths) float64 1e-50 ... 1e-50
    feh           (pressures, temperatures, wavelengths) float64 6.94e-25 ......
    ...            ...
    Lupu_alk      (pressures, temperatures, wavelengths) float64 1.604e-19 .....
    n2            (pressures, temperatures, wavelengths) float64 5.145e-31 .....
    nh3           (pressures, temperatures, wavelengths) float64 2.489e-22 .....
    ph3           (pressures, temperatures, wavelengths) float64 1.77e-23 ......
    tio           (pressures, temperatures, wavelengths) float64 3.825e-21 .....
    vo            (pressures, temperatures, wavelengths) float64 1e-50 ... 6....

In [21]:
from scipy.interpolate import PchipInterpolator as monotonic_interpolation
from scipy.ndimage import gaussian_filter1d as gaussian_smoothing

In [22]:
def piette(T_m4, T_m3, T_m2, T_m1, T_0, T_0p5, T_1, T_1p5, T_2, T_2p5,
           num_layers_final=71, P_min=-4, P_max=2.5):
    logP_nodes = np.array([-4, -3, -2, -1, 0, 0.5, 1, 1.5, 2])
    T_nodes = np.array([T_m4, T_m3, T_m2, T_m1, T_0, T_0p5, T_1, T_1p5, T_2])

    interpolated_function = monotonic_interpolation(logP_nodes, T_nodes)
    input_Ps = np.linspace(P_min, P_max, num=num_layers_final)

    profile = gaussian_smoothing(interpolated_function(input_Ps), sigma=0.3)

    return profile

In [27]:
Ps = np.linspace(-4, 2.5, num=71)
Ts = piette(697, 769, 802, 914, 967, 968, 1021, 1096, 1102, 1119)

In [28]:
import chunks_as_classes as cac
from pathlib import Path
from matplotlib import pyplot as plt

In [29]:
test_data_file = Path("/Users/arthur/Documents/Astronomy/2019/Retrieval/Code/Data/mock-L/mock-L.2022-12-08.forward-model.PLO.JHK.noised.dat")
band_names = ["J", "H", "Ks"]

In [33]:
JHK_data = cac.read_APOLLO_data(test_data_file, band_names)

In [36]:
waves = JHK_data["J"].wavelengths[1:]

In [37]:
waves

array([1.10029 , 1.100435, 1.10058 , ..., 1.3536  , 1.353745, 1.35389 ])

In [41]:
opacities_interpd = opacity_Dataset.interp(
    pressures=Ps,
    temperatures=Ts,
    wavelengths=waves,
    assume_sorted=True
)

In [42]:
opacities_interpd

<xarray.Dataset>
Dimensions:       (pressures: 71, temperatures: 71, wavelengths: 1750)
Coordinates:
  * pressures     (pressures) float64 -4.0 -3.907 -3.814 ... 2.314 2.407 2.5
  * temperatures  (temperatures) float64 697.0 705.4 ... 1.097e+03 1.094e+03
  * wavelengths   (wavelengths) float64 1.1 1.1 1.101 ... 1.354 1.354 1.354
Data variables: (12/19)
    Burrows_alk   (pressures, temperatures, wavelengths) float64 nan ... 1.23...
    ch4           (pressures, temperatures, wavelengths) float64 nan ... 9.66...
    co            (pressures, temperatures, wavelengths) float64 nan ... 1e-50
    co2           (pressures, temperatures, wavelengths) float64 nan ... 3.17...
    crh           (pressures, temperatures, wavelengths) float64 nan ... 1e-50
    feh           (pressures, temperatures, wavelengths) float64 nan ... 4.83...
    ...            ...
    Lupu_alk      (pressures, temperatures, wavelengths) float64 nan ... 1.71...
    n2            (pressures, temperatures, wavelengths) float64 nan ... 1e-50
    nh3           (pressures, temperatures, wavelengths) float64 nan ... 1.21...
    ph3           (pressures, temperatures, wavelengths) float64 nan ... 8.76...
    tio           (pressures, temperatures, wavelengths) float64 nan ... 2.55...
    vo            (pressures, temperatures, wavelengths) float64 nan ... 1e-50

In [40]:
nc_interpd_save_file = "nir_opacities_test_interp_to_data.nc"
opacities_interpd.to_netcdf(nc_interpd_save_file)